In [1]:
import pickle
from PIL import Image
import numpy as np
from tqdm import tqdm

target_size = 256

In [2]:
## 피클파일 없을 때 실행

# 메타데이터 및 텍스트 데이터 로드

train_path = './train_data/%s'
metadata = pickle.load(open('./data/sentence_gamjung2.pkl', 'rb'))

# clean_text에 시작, 끝 추가
# clean_text 기준 256글자 넘지 않도록 필터링
# 각 감정 클래스가 최대한 균등하도록 샘플 추출
# 학습 5000, 검증 1000, 테스트 1000

metadata['clean_text'] = metadata['clean_text'].apply(lambda x: '\t%s\n' %x.strip())
metadata = metadata[metadata['clean_text'].str.len() <= 256]

sentiment_list = ['emotion_0', 'emotion_1', 'emotion_3', 'emotion_4']
sentiment_indices = [metadata[metadata[senti] == 1].index for senti in sentiment_list]
data_indices = []

i = 0
condition = True
while condition:
    for senti in sentiment_indices:
        if len(set(data_indices)) < 7000:
            data_indices.append(senti[i])
        else:
            condition = False
            break
    i += 1
    
metadata = metadata.iloc[sorted(list(set(data_indices)))]

def get_image_from_path(path):
    image = Image.open(train_path %path)
    size = image.size

    if size[0] <= size[1]:
        scaled_size = (target_size, int(size[1]/size[0]*target_size))
        crop_area = (0, int((scaled_size[1]-target_size)/2), target_size, int((scaled_size[1]+target_size)/2))
    else:
        scaled_size = (int(size[0]/size[1]*target_size), target_size)
        crop_area = (int((scaled_size[0]-target_size)/2), 0, int((scaled_size[0]+target_size)/2), target_size)

    image = image.resize(scaled_size)
    image = image.crop(crop_area)
    return image

def get_array_from_path(path):
    return np.asarray(get_image_from_path(path))

# 이미지 불러와서 데이터셋 생성
image_data = np.array([image for image in metadata['image_path'].map(get_array_from_path)])
sentiment_data = np.array(metadata[sentiment_list])
sentence_data = np.array(metadata['clean_text'])

# 데이터 피클로 저장
pickle.dump([image_data, sentiment_data, sentence_data], open('data.pkl', 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: './data/sentence_gamjung2.pkl'

In [3]:
# 피클로 저장된 데이터 로드
image_data, sentiment_data, sentence_data = pickle.load(open('data.pkl', 'rb'))

In [4]:
print(image_data.shape)
print(sentiment_data.shape)
print(sentence_data.shape)

(7000, 256, 256, 3)
(7000, 4)
(7000,)


In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tf.reset_default_graph()

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.build(input_shape=(None, target_size, target_size, 3))
model.summary()